## Basic of GCNN

1.GGNN通过隐层 hidden layer 来传递信息；

2.GCNN累积前层网络的信息并且产生图中节点的有用的特征表达 feature representation of node;

GCNN是操作在图结构数据上的深层神经网络，其输入input为：

* $N*F^0$的输入特征矩阵$X$,$N$是节点个数，$F^0$是每一个节点的输入特征的数量；
* 可以用$N*N$邻接矩阵Adjacent matrix $A$ 来表达图结构；

GCNN的hidden layer可以被表示为:$H^i=f((H^{i-1}, A))$, 其中$H^0=X$，$f$是非线性激活函数ReLU[reference](https://tkipf.github.io/graph-convolutional-networks/),每一个隐层$H^i$是$N * F^i$的特征矩阵，每一行是一个节点的特征表达。下一层的特征由累积的前层特征经过$f$函数映射得到。这样随着网络层数以及网络相关性的增加，特征会变得越来越抽象(高维特征)。一般不同的GCN体现在传播规则$f$的不同上。

## 简单Propagation Rule的一个例子

一个最简单的例子就是采用非线性激活函数[ReLU](https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29) :
$$f(H^i, A)=\sigma (AH^iW^i)$$
其中$W^i$为第$i$层的权重矩阵，$\sigma$是非线性激活函数ReLU,权重矩阵的维度为$F^i * F^{i+1}$，可知第二维$F^{i+1}$决定了下一层网络的特征的数量，而$H^i$维度为$N*F^i$的矩阵，A为$N*N$的邻接矩阵，因此相当于对$N*F^{i+1}$维的特征进行$\sigma$非激活函数映射。此处和[卷积神经网络](https://en.wikipedia.org/wiki/Convolutional_neural_network)中卷积操作非常相似，[feference](卷积神经网络基础操作),因为在图结构数据的节点中权重是共享的。


## 图结构的一个简单例子：

如下有向图图：

![一个简单的有向图](https://cdn-images-1.medium.com/max/800/1*jTW7doI_cqC_p9XQrmuu9A.png)

上图可由numpy矩阵表示为：

In [42]:
import numpy as np
A=np.matrix([[0,1,0,0],
             [0,0,1,1],
             [0,1,0,0],
             [1,0,1,0]], dtype=float)
A

matrix([[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  1.],
        [ 0.,  1.,  0.,  0.],
        [ 1.,  0.,  1.,  0.]])

在式$f(H^i, A)=\sigma(AH^iW^i)$中定义输入特征$H^i$,维度为$N*F^{i+1}$，令$F^{i+1}$为2，则可表示为：

In [24]:
import numpy as np
H = np.matrix([[i, -i] for i in range(A.shape[0])], dtype=float)
H

matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

现在让$W^i$为单位矩阵，令$f(H^i, A)= \sigma(AH^iW^i)=AH^i$,可发现，矩阵乘操作后的结果为节点临边的节点信息的和，则：

In [36]:
H_=A*H 
H_

matrix([[ 1., -1.],
        [ 5., -5.],
        [ 1., -1.],
        [ 2., -2.]])

可以发现，上图不存在自己指向自己的边，我们可以通过向$A$矩阵添加一个对应的单位矩阵$A\_E$来实现,相当于添加了将自己作为临边的节点信息：

In [43]:
A_E = np.matrix(np.eye(A.shape[0]))
A_E

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]])

In [44]:
A_=A+A_E
A_

matrix([[ 1.,  1.,  0.,  0.],
        [ 0.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  1.,  1.]])

### 规则化特征表达 Normalization

$f(X,A)=D^{-1}AX$ ,其中$D^{-1}$为矩阵A行向量量级构成的对角阵的逆 the inverse degree matrix of D, 如下所示：

In [53]:
D=np.array(np.sum(A, axis=0))[0] #np.sum(A, axis=0).shape =(1, 4), a 2 dimention matrixs
D = np.matrix(np.diag(D)) #make the degree matrix of A
A = D**-1 * A
A

matrix([[ 0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  0.5,  0.5],
        [ 0. ,  0.5,  0. ,  0. ],
        [ 1. ,  0. ,  1. ,  0. ]])

In [56]:
A_=A*H
A_

matrix([[ 1. , -1. ],
        [ 2.5, -2.5],
        [ 0.5, -0.5],
        [ 2. , -2. ]])

## Other

另外，可以在上式A_进行非线性激活函数ReLU操作之前乘上权重层$W^i$，其维度为$F^{i+1}*F^{i+1}$。